In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device= 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size=8
batch_size=8
learning_rate= 3e-4
max_iters = 100000
eval_iters = 250

cuda


In [2]:
with open('wizard_of_oz.txt','r',encoding='utf-8') as f:
    text = f.read()
    chars=sorted(set(text))
vocab_size = len(chars)

In [3]:
string_to_int={ch:i for i,ch in enumerate(chars)}#here we did mapping 
int_to_string={i:ch for i,ch in enumerate(chars)}
encode=lambda s:[string_to_int[c] for c in s] # lambda arguments: expression
decode=lambda l:''.join([int_to_string[i] for i in l ])

In [4]:
print(encode(text[:100]))

[91, 48, 65, 62, 1, 44, 75, 72, 67, 62, 60, 77, 1, 35, 78, 77, 62, 71, 59, 62, 75, 64, 1, 62, 30, 72, 72, 68, 1, 72, 63, 1, 32, 72, 75, 72, 77, 65, 82, 1, 58, 71, 61, 1, 77, 65, 62, 1, 51, 66, 83, 58, 75, 61, 1, 66, 71, 1, 43, 83, 0, 1, 1, 1, 1, 0, 48, 65, 66, 76, 1, 62, 59, 72, 72, 68, 1, 66, 76, 1, 63, 72, 75, 1, 77, 65, 62, 1, 78, 76, 62, 1, 72, 63, 1, 58, 71, 82, 72, 71]


In [5]:
print(decode([91, 48, 65, 62, 1, 44, 75, 72, 67, 62, 60, 77, 1, 35, 78, 77, 62, 71, 59, 62, 75, 64, 1, 62, 30, 72, 72, 68, 1, 72, 63, 1, 32, 72, 75, 72, 77, 65, 82, 1, 58, 71, 61, 1, 77, 65, 62, 1, 51, 66, 83, 58, 75, 61, 1, 66, 71, 1, 43, 83, 0, 1, 1, 1, 1, 0, 48, 65, 66, 76, 1, 62, 59, 72, 72, 68, 1, 66, 76, 1, 63, 72, 75, 1, 77, 65, 62, 1, 78, 76, 62, 1, 72, 63, 1, 58, 71, 82, 72, 71]
))

﻿The Project Gutenberg eBook of Dorothy and the Wizard in Oz
    
This ebook is for the use of anyon


In [6]:
data=torch.tensor(encode(text),dtype=torch.long) # jo mapped charcters hai unko hamne ek tensor(kinda matrix) pe store kiya as of now
print(data[:100])

tensor([91, 48, 65, 62,  1, 44, 75, 72, 67, 62, 60, 77,  1, 35, 78, 77, 62, 71,
        59, 62, 75, 64,  1, 62, 30, 72, 72, 68,  1, 72, 63,  1, 32, 72, 75, 72,
        77, 65, 82,  1, 58, 71, 61,  1, 77, 65, 62,  1, 51, 66, 83, 58, 75, 61,
         1, 66, 71,  1, 43, 83,  0,  1,  1,  1,  1,  0, 48, 65, 66, 76,  1, 62,
        59, 72, 72, 68,  1, 66, 76,  1, 63, 72, 75,  1, 77, 65, 62,  1, 78, 76,
        62,  1, 72, 63,  1, 58, 71, 82, 72, 71])


In [7]:
n=int(0.8*len(data))
train_data=data[:n]
val_data=data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    # print(ix)
    x = torch.stack([data[i:i+block_size]for i in ix])
    y = torch.stack([data[i+1:i+1+block_size]for i in ix])
    x,y = x.to(device) , y.to(device)
    return x,y
x,y = get_batch('train')

print('inputs')
print(x)
print('targets')
print(y)

inputs
tensor([[66, 71, 64,  1, 58, 71, 61,  0],
        [69, 66, 68, 62, 69, 82,  1, 77],
        [78, 76, 69, 82,  1, 58, 69, 72],
        [70, 66, 71, 78, 77, 62, 27,  1],
        [ 1, 72, 75,  1, 69, 66, 77, 77],
        [ 1, 79, 62, 75, 82,  1, 72, 69],
        [12,  1, 76, 72,  1, 77, 65, 58],
        [77, 76,  1, 80, 62, 75, 62,  1]], device='cuda:0')
targets
tensor([[71, 64,  1, 58, 71, 61,  0, 75],
        [66, 68, 62, 69, 82,  1, 77, 72],
        [76, 69, 82,  1, 58, 69, 72, 71],
        [66, 71, 78, 77, 62, 27,  1, 59],
        [72, 75,  1, 69, 66, 77, 77, 69],
        [79, 62, 75, 82,  1, 72, 69, 61],
        [ 1, 76, 72,  1, 77, 65, 58, 77],
        [76,  1, 80, 62, 75, 62,  1, 58]], device='cuda:0')


In [8]:

x=train_data[:block_size] # 0 to 7
y=train_data[1:block_size+1] # 1 to 8
for t in range (block_size): # 0 to 7
    context=x[:t+1] # 0 to t
    target=y[t] # tth 
    print('input is tensor',context,'target is ',target)

input is tensor tensor([91]) target is  tensor(48)
input is tensor tensor([91, 48]) target is  tensor(65)
input is tensor tensor([91, 48, 65]) target is  tensor(62)
input is tensor tensor([91, 48, 65, 62]) target is  tensor(1)
input is tensor tensor([91, 48, 65, 62,  1]) target is  tensor(44)
input is tensor tensor([91, 48, 65, 62,  1, 44]) target is  tensor(75)
input is tensor tensor([91, 48, 65, 62,  1, 44, 75]) target is  tensor(72)
input is tensor tensor([91, 48, 65, 62,  1, 44, 75, 72]) target is  tensor(67)


In [9]:
#It is checking how well the model is doing (the loss) on both the training and validation data — but without changing the model. that is without optimizing
@torch.no_grad()
def estimate_loss():
    out={}
    model.eval()
    for split in ['train','val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            x,y= get_batch(split)
            logits,loss = model.forward(x,y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out
    

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
                                                                                                                                                                                          
class BigramLanguageModel(nn.Module): # nn.module se inherit kar rahe hai chize
    def __init__(self, vocab_size): # init predefined chiz hai to initialize the BLM
        super().__init__() # to initialize nn.module ye basically nn.module (parent class) ke constructor ko call karta hai
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size) # self.jaruri hai ye ek predefined chiz hai jo ek class ki khud ki chize store karwane and infact access karwane me help karta hai
    
    def forward(self, idx, targets=None): # ab ham learnable parameters ko train karenge ....
        logits = self.token_embedding_table(idx) # (basically here we played smart and did vector embeddings = logits)(tabhi embedding table vocab*vocab ka bana hai)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape # common parameters in ML b = batch size, T = time/sequence length, C = vocab size
            logits = logits.view(B*T, C) # .view(), it's used to reshape a tensor without copying data.
            targets = targets.view(B*T) # total number of tokens = B*T
            loss = F.cross_entropy(logits, targets) 
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens): # Use underscore for unused variable
            logits, loss = self.forward(idx)
            logits = logits[:, -1, :] # negative indexing on block size (T) - get last time step
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx


model = BigramLanguageModel(vocab_size)
m = model.to(device)
context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


rL*Yq36_/R(﻿‘qyGDN1(mi%1l$RemAIZWmuDM“_wNk2F’W$•tH™I—D'WWUsAPFW•“#8—nK9mqx﻿1,nu9*o5t-Ajg,"#f”﻿ •r"vOE—,A)E-_AC4$CQ3.-2‘L3CEf W$CK•_.u'—“]S0lr-__BmDpn(]aMM[#s”3rLE0]c(H™qdDyw$u%F2—]gos,oWE;6
gad2JoXBBlLFRw-8”bWeJkpJ5)aZZ_OFW$!?“exp&]GkH™Ma‘p(l”,‘•t]4Ih*6(Z%k [p6]Y0L'WU”IR&Y—b9s)cVd44!‘H9asd0rNhY[gd9n'Ny(T3 !?PM
[vCQDfIj:”
TtVMa#;H*x78H.]JkpcDVDd.]2a
(Q49.T9.d—IRVLj#3T—]bPTsYFvjy&wt-UF—YEFH“IRg—Ax6”
)Yh;5c98N;a7;Qgq3ytd“
0r[ SEZqkc.”‘U*gdh(’Lii-“L
8Pf!VF,IRSAAb5xN6mY2i$JG,DI'M;rNxyTka_CE‘y—3rRzcTg


In [11]:
optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)
for iter in range (max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f'step = : {iter} ,loss {losses}')
    xb , yb = get_batch('train')
    logits,loss = model.forward(xb,yb)
    optimizer.zero_grad(set_to_none = True) # hmm this step is just cleaning the mess for backpropagation
    loss.backward() # back propagation 
    optimizer.step() # optimizing that is tuning in the opposite direction of gradient calculated by back prpagation
print(loss.item()) # loss of the last iteration 

step = : 0 ,loss {'train': tensor(4.9889), 'val': tensor(4.9879)}
step = : 250 ,loss {'train': tensor(4.9063), 'val': tensor(4.9164)}
step = : 500 ,loss {'train': tensor(4.8495), 'val': tensor(4.8260)}
step = : 750 ,loss {'train': tensor(4.7670), 'val': tensor(4.7680)}
step = : 1000 ,loss {'train': tensor(4.7027), 'val': tensor(4.6935)}
step = : 1250 ,loss {'train': tensor(4.6087), 'val': tensor(4.6330)}
step = : 1500 ,loss {'train': tensor(4.5606), 'val': tensor(4.5696)}
step = : 1750 ,loss {'train': tensor(4.4875), 'val': tensor(4.4995)}
step = : 2000 ,loss {'train': tensor(4.4390), 'val': tensor(4.4284)}
step = : 2250 ,loss {'train': tensor(4.3625), 'val': tensor(4.3837)}
step = : 2500 ,loss {'train': tensor(4.3086), 'val': tensor(4.3145)}
step = : 2750 ,loss {'train': tensor(4.2418), 'val': tensor(4.2409)}
step = : 3000 ,loss {'train': tensor(4.1703), 'val': tensor(4.2014)}
step = : 3250 ,loss {'train': tensor(4.1355), 'val': tensor(4.1457)}
step = : 3500 ,loss {'train': tensor(4.0

In [12]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)



Jid Wit, thefuswatotre r, icatwit.

l, pp sthe e th
cowe azat k lileshedol?" be wony l ind d athe bubrorind ARoutens trcim as, weaps?" ine opthtorous

ws
thet we ale wine t ar wlle brd."Alsm f aned weallory whenghed s, t e alo fr tts ZMaly l poraith asss n dove s Jon mair

t alit aiganourooft wiowein ckin's meryome asif ouedagleer th Jiou Pe M y  herraind l imuthad ne the orod herrave tlene.
"HEI m, ins." I thur cy as," h wicans whather as we freser h; t the
grutheedotheace t Dond e oche cavab.
